In [79]:
from fetchYahooStockData import metadata_price_action, extended_stock_stats, basic_stats, current_sp500_symbols, parse_value, current_dow30_symbols
import numpy as np
import pandas as pd

In [90]:
symbol="TSLA"
timeInterval = "1m"
longTimeInterval = "1h"
timeRange = "5d"

In [91]:
metadataAndPrice = metadata_price_action(symbol, timeInterval, timeRange)
metadata = metadataAndPrice["metadata"]
metadata

{'currency': 'USD',
 'symbol': 'TSLA',
 'exchangeName': 'NMS',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 1277818200,
 'regularMarketTime': 1614373202,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 675.5,
 'chartPreviousClose': 781.3,
 'previousClose': 682.22,
 'scale': 3,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614330000,
   'end': 1614349800,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614349800,
   'end': 1614373200,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614373200,
   'end': 1614387600,
   'gmtoffset': -18000}},
 'tradingPeriods': [[{'timezone': 'EST',
    'start': 1614004200,
    'end': 1614027600,
    'gmtoffset': -18000}],
  [{'timezone': 'EST',
    'start': 1614090600,
    'end': 1614114000,
    'gmtoffset': -18000}],
  [{'timezone': 'EST',
    'start': 1614177000,
    'end': 1614200400,
    'gmtoffset': -18000}]

In [92]:
longMetadataAndPrice = metadata_price_action(symbol, longTimeInterval, timeRange)
longMetadata = metadataAndPrice["metadata"]
longMetadata

{'currency': 'USD',
 'symbol': 'TSLA',
 'exchangeName': 'NMS',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 1277818200,
 'regularMarketTime': 1614373202,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 675.5,
 'chartPreviousClose': 781.3,
 'previousClose': 682.22,
 'scale': 3,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614330000,
   'end': 1614349800,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614349800,
   'end': 1614373200,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614373200,
   'end': 1614387600,
   'gmtoffset': -18000}},
 'tradingPeriods': [[{'timezone': 'EST',
    'start': 1614004200,
    'end': 1614027600,
    'gmtoffset': -18000}],
  [{'timezone': 'EST',
    'start': 1614090600,
    'end': 1614114000,
    'gmtoffset': -18000}],
  [{'timezone': 'EST',
    'start': 1614177000,
    'end': 1614200400,
    'gmtoffset': -18000}]

In [93]:
price = metadataAndPrice["price"]
price.columns = ["timestamp", "Close", "Open", "High", "Volume", "Low"]
price = price.dropna()
price

,timestamp,Close,Open,High,Volume,Low
0,1614004200,758.000000,762.640015,762.770020,1723744.0,761.099976
1,1614004260,755.500000,761.140198,761.479980,201050.0,755.580017
2,1614004320,755.150024,755.869873,758.998230,186583.0,756.099976
3,1614004380,751.130005,756.395020,758.000000,299695.0,753.539978
4,1614004440,754.325012,754.395020,754.739990,215484.0,754.539978
...,...,...,...,...,...,...
1945,1614372900,678.260010,679.200012,680.380005,201650.0,678.969971
1946,1614372960,677.682129,678.969971,680.375977,155722.0,678.984985
1947,1614373020,678.380005,679.049988,680.049988,126658.0,679.275024
1948,1614373080,678.179993,679.275024,679.539978,225960.0,678.630005


In [94]:
longPrice = longMetadataAndPrice["price"]
longPrice.columns = ["timestamp", "Close", "Open", "High", "Volume", "Low"]
longPrice

,timestamp,Close,Open,High,Volume,Low
0,1614004200,752.949219,762.640015,768.320007,751.130005,8400327
1,1614007800,744.979980,752.950012,754.940002,735.000000,8119527
2,1614011400,741.049988,744.979980,749.900024,739.500000,2545407
3,1614015000,743.219971,741.150024,746.091003,739.358582,2134844
4,1614018600,738.000000,743.150024,745.390015,737.739990,1934571
5,1614022200,720.905029,737.770020,738.530029,719.549988,5468306
6,1614025800,714.710022,721.099976,723.500000,710.200012,6510511
7,1614090600,691.270020,662.130005,694.239990,619.000000,29830758
8,1614094200,687.130005,691.419983,708.989990,680.049988,11994217
9,1614097800,681.253418,687.049988,693.306702,668.820007,6067487


In [110]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from statistics import mean

def mySMA(myArr, period):
    return pd.Series(myArr).rolling(period).mean()
            
def myEMA(myArr, period):
    return pd.Series(myArr).ewm(adjust=False, span=period).mean()
    
def myMacd(myArr):
    twelve = myEMA(myArr, 12) 
    twentySix = myEMA(myArr, 26)
    macd = twelve-twentySix
#     print(macd, twelve, twentySix)
    return macd

def myMacdSignal(myArr):
    return myEMA(myMacd(myArr), 9)

def myMacdHist(myArr):
    return myMacd(myArr) - myMacdSignal(myArr)
    
class SmaCross(Strategy):
    def init(self):
        tempprice = self.data.Close
        self.ma1 = self.I(myEMA, tempprice, 8)
        self.ma2 = self.I(myEMA, tempprice, 13)
        self.macd = self.I(myMacd, tempprice)
        self.macdSignal = self.I(myMacdSignal, tempprice)
        self.macdHist = self.I(myMacdHist, tempprice)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(price, SmaCross,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

<ipython-input-110-c7e435f6e94f>:41: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(price, SmaCross,


In [87]:
basicStats = basic_stats(symbol)
basicStats

{'Previous Close': '82.42',
 'Open': '83.57',
 'Bid': '84.30 x 900',
 'Ask': '84.42 x 800',
 "Day's Range": '82.91 - 85.59',
 '52 Week Range': '36.75 - 99.23',
 'Volume': '48,962,907',
 'Avg. Volume': '43,362,416',
 'Market Cap': '102.369B',
 'Beta (5Y Monthly)': '2.20',
 'PE Ratio (TTM)': '40.94',
 'EPS (TTM)': '2.06',
 'Earnings Date': 'Apr 26, 2021 - Apr 30, 2021',
 'Forward Dividend & Yield': 'N/A (N/A)',
 'Ex-Dividend Date': 'Apr 27, 1995'}

In [88]:
extStats = extended_stock_stats(symbol)
extStats

{'Market Cap (intraday) 5': ['102.65B',
  '111.16B',
  '98.55B',
  '61.76B',
  '53.26B',
  '53.66B'],
 'Enterprise Value 3': ['100.89B',
  '109.97B',
  '97.67B',
  '61.08B',
  '52.48B',
  '53.56B'],
 'Trailing P/E ': ['41.14', '123.93', '157.67', '122.35', '151.60', '241.37'],
 'Forward P/E 1': ['44.44', '51.02', '49.26', '52.63', '39.84', '43.48'],
 'PEG Ratio (5 yr expected) 1': ['1.58',
  '1.13',
  '1.42',
  '1.55',
  '1.66',
  '1.55'],
 'Price/Sales (ttm)': ['10.48', '12.68', '12.67', '8.37', '7.57', '8.38'],
 'Price/Book (mrq)': ['17.59', '28.72', '29.82', '20.34', '18.84', '24.66'],
 'Enterprise Value/Revenue 3': ['10.33',
  '33.90',
  '34.87',
  '31.61',
  '29.39',
  '25.18'],
 'Enterprise Value/EBITDA 6': ['60.20',
  '167.63',
  '193.41',
  '238.59',
  '202.63',
  '180.94'],
 'Beta (5Y Monthly) ': '2.20',
 '52-Week Change 3': '78.07%',
 'S&P500 52-Week Change 3': '16.75%',
 '52 Week High 3': '99.23',
 '52 Week Low 3': '36.75',
 '50-Day Moving Average 3': '89.89',
 '200-Day Movi